<a href="https://colab.research.google.com/github/alexmal2804/hwds/blob/master/HW_%22Part_1_Intro_to_SQL_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1MSg54lxqkX_1910LKrFHRJnAqH_a8IF8?usp=sharing

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [ ]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [ ]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [ ]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
os.path.exists(path)

True

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [ ]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [ ]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [ ]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [ ]:
conn.commit()
conn.close()

In [ ]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [ ]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [ ]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [ ]:
cursor.close()
conn.close()

In [ ]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [ ]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [ ]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [ ]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [ ]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50
6,Real Madrid,53
7,Barcelona,47
8,Arsenal,52
9,Real Madrid,49


In [ ]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [ ]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [ ]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-10 07:46:27,488 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-10 07:46:27,489 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:27,490 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-10 07:46:27,492 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [ ]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [ ]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4546 sha256=d28b4567f18952552201463308b10b6382e4c0fdb903f0fd31ed83228287af47
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [ ]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [ ]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [ ]:
Purchase.ItemId.name

'ItemId'

In [ ]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [ ]:
Base.metadata.create_all(engine)

2020-12-10 07:46:31,681 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-10 07:46:31,682 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,684 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-10 07:46:31,685 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,686 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-10 07:46:31,687 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,689 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-10 07:46:31,693 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,694 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-10 07:46:31,696 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,697 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-10 07:46:31,698 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:31,699 INFO sqlalchemy.engine.base.Engine 
CREATE

In [ ]:
engine.table_names()

2020-12-10 07:46:31,722 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-10 07:46:31,723 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [ ]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-10 07:46:31,747 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:31,749 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 07:46:31,750 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-10 07:46:31,753 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [ ]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-10 07:46:31,766 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-10 07:46:31,768 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [ ]:
display_results(customers_query)

2020-12-10 07:46:31,780 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-10 07:46:31,782 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [ ]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-10 07:46:31,832 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:31,834 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 07:46:31,836 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-10 07:46:31,837 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-10 07:46:31,851 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:31,852 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 07:46:31,854 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-10 07:46:31,855 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-10 07:46:31,874 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:31,876 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 07:46:31,877 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-10 07:46:31,879 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [ ]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-10 07:46:31,889 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-10 07:46:31,890 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [ ]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a410>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a468>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a5c8>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a0f8>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a1a8>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a4c0>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fbfb788a3b8>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7fbfd31ad6d8; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [ ]:
from sqlalchemy import *

In [ ]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-10 07:46:31,942 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-10 07:46:31,944 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [ ]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-10 07:46:31,973 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-10 07:46:31,975 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-10 07:46:32,009 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-10 07:46:32,011 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

In [ ]:
display_results(customers_query)

2020-12-10 07:46:32,047 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-10 07:46:32,048 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [ ]:
import pandas as pd

In [ ]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [ ]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-10 07:46:32,095 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-10 07:46:32,096 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,099 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-10 07:46:32,100 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,102 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-10 07:46:32,103 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [ ]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-10 07:46:32,149 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 07:46:32,151 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,153 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 07:46:32,158 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,160 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-10 07:46:32,162 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [ ]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-10 07:46:32,200 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 07:46:32,201 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,204 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 07:46:32,205 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 07:46:32,207 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-10 07:46:32,209 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

##Задание 1. 
Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач.

<center><img src='https://github.com/alexmal2804/hwds/blob/master/hw_sql_picture2.jpg?raw=true'></center>


##Задание 2
Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.

class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [ ]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
#engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from sqlalchemy import select
customers_query = select([Customer.CustomerId, Customer.FirstName, Customer.LastName, Customer.Email, Customer.Address, Customer.Company, Customer.Phone])
display_results(customers_query)

2020-12-10 07:46:32,244 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."CustomerId", customers."FirstName", customers."LastName", customers."Email", customers."Address", customers."Company", customers."Phone" 
FROM customers
2020-12-10 07:46:32,245 INFO sqlalchemy.engine.base.OptionEngine ()


,CustomerId,FirstName,LastName,Email,Address,Company,Phone
0,1,Moshe,Cohen,moshe@cohen.com,"Alenbi 99, Tel Aviv",None,053-5556789
1,2,Lisa,Cohen,lisa@cohen.com,"Alenbi 99, Tel Aviv",None,052-1234565
2,3,Nika,Rave,Nika@rave.com,"Green st, LA",None,330-1234565
3,4,Lisa,White,lisa@White.com,"Alenbi 66, Tel Aviv",None,062-1234565


In [ ]:
Knife = Item(
    Name = 'Knife',
    Price = 124
    )
session.add(Knife)

Fork = Item(
    Name = 'Fork',
    Price = 75
)
session.add(Fork)

Spoon = Item(
    Name = 'Spoon',
    Price = 87
)
session.add(Spoon)

Plate = Item(
    Name = 'Plate',
    Price = 93
)
session.add(Plate)

Candlestick = Item(
    Name = 'Candlestick',
    Price = 178
)
session.add(Candlestick)

session.commit()

2020-12-10 07:46:32,283 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:32,284 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 07:46:32,286 INFO sqlalchemy.engine.base.Engine ('Knife', 124.0)
2020-12-10 07:46:32,288 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 07:46:32,289 INFO sqlalchemy.engine.base.Engine ('Fork', 75.0)
2020-12-10 07:46:32,290 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 07:46:32,291 INFO sqlalchemy.engine.base.Engine ('Spoon', 87.0)
2020-12-10 07:46:32,293 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 07:46:32,293 INFO sqlalchemy.engine.base.Engine ('Plate', 93.0)
2020-12-10 07:46:32,296 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 07:46:32,296 INFO sqlalchemy.engine.base.Engine ('Candlestick', 178.

In [ ]:
items_query = select([Item.ItemId, Item.Name, Item.Price])
display_results(items_query)

2020-12-10 07:46:32,306 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId", items."Name", items."Price" 
FROM items
2020-12-10 07:46:32,307 INFO sqlalchemy.engine.base.OptionEngine ()


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,ItemId,Name,Price
0,1,Knife,124.0
1,2,Fork,75.0
2,3,Spoon,87.0
3,4,Plate,93.0
4,5,Candlestick,178.0


In [ ]:
import datetime

In [ ]:
purchase1_1 = Purchase(
    ItemId = 1,
    CustomerId = 1,
    Date = datetime.datetime(2020, 8, 12)
)
session.add(purchase1_1)
purchase2_1 = Purchase(
    ItemId = 4,
    CustomerId = 1,
    Date = datetime.datetime(2019, 12, 30)
)
session.add(purchase2_1)
purchase2_1 = Purchase(
    ItemId = 5,
    CustomerId = 2,
    Date = datetime.datetime(2020, 10, 25)
)
session.add(purchase2_1)
purchase2_2 = Purchase(
    ItemId = 3,
    CustomerId = 2,
    Date = datetime.datetime(2020, 2, 23)
)
session.add(purchase2_2)
purchase4_1 = Purchase(
    ItemId = 5,
    CustomerId = 4,
    Date = datetime.datetime(2019, 10, 8)
)
session.add(purchase4_1)
session.commit()

2020-12-10 07:46:32,367 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 07:46:32,369 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 07:46:32,370 INFO sqlalchemy.engine.base.Engine (1, 1, '2020-08-12 00:00:00.000000')
2020-12-10 07:46:32,372 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 07:46:32,373 INFO sqlalchemy.engine.base.Engine (4, 1, '2019-12-30 00:00:00.000000')
2020-12-10 07:46:32,374 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 07:46:32,376 INFO sqlalchemy.engine.base.Engine (5, 2, '2020-10-25 00:00:00.000000')
2020-12-10 07:46:32,377 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 07:46:32,378 INFO sqlalchemy.engine.base.Engine (3, 2, '2020-02-23 00:00:00.000000')
2020-12-10 07:46

In [ ]:
purchase_query = select([Purchase.PurchaseId, Purchase.ItemId, Purchase.CustomerId, Purchase.Date])
display_results(purchase_query)

2020-12-10 07:48:18,487 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."PurchaseId", purchases."ItemId", purchases."CustomerId", purchases."Date" 
FROM purchases
2020-12-10 07:48:18,490 INFO sqlalchemy.engine.base.OptionEngine ()


,PurchaseId,ItemId,CustomerId,Date
0,1,1,1,2020-08-12
1,2,4,1,2019-12-30
2,3,5,2,2020-10-25
3,4,3,2,2020-02-23
4,5,5,4,2019-10-08


## Вопрос 3.
При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

In [ ]:
Lisa_query = select([Customer.FirstName, Customer.LastName, Item.Name, Item.Price, Purchase.Date]).where(and_(Customer.CustomerId == Purchase.CustomerId, Purchase.ItemId == Item.ItemId, Customer.FirstName == 'Lisa'))
display_results(Lisa_query)

2020-12-10 08:50:36,313 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", items."Name", items."Price", purchases."Date" 
FROM customers, items, purchases 
WHERE customers."CustomerId" = purchases."CustomerId" AND purchases."ItemId" = items."ItemId" AND customers."FirstName" = ?
2020-12-10 08:50:36,314 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Name,Price,Date
0,Lisa,Cohen,Candlestick,178.0,2020-10-25
1,Lisa,Cohen,Spoon,87.0,2020-02-23
2,Lisa,White,Candlestick,178.0,2019-10-08


In [ ]:
sql = """
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';
"""
df = pd.read_sql(sql, conn)
df

2020-12-10 09:11:22,923 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';
")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';
")


2020-12-10 09:11:22,926 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-10 09:11:22,928 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';
")


...

2020-12-10 09:11:22,934 INFO sqlalchemy.engine.base.Engine 
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';



INFO:sqlalchemy.engine.base.Engine:
SELECT Customers.FirstName, Customers.LastName, Items.Name, Items.Price, Purchases.Date 
FROM Customers, Items, Purchases
WHERE  Customers.CustomerId = Purchases.CustomerId and Purchases.ItemId = Items.ItemId and Customers.FirstName = 'Lisa';



2020-12-10 09:11:22,936 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,FirstName,LastName,Name,Price,Date
0,Lisa,Cohen,Candlestick,178,2020-10-25 00:00:00.000000
1,Lisa,Cohen,Spoon,87,2020-02-23 00:00:00.000000
2,Lisa,White,Candlestick,178,2019-10-08 00:00:00.000000
